In [4]:
import numpy as np
import pandas as pd
# Using regex
import re
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
# fix random seed for reproducibility
np.random.seed(7)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def load_as_list(fname):
    df = pd.read_csv(fname)
    id = df['id'].values.tolist()
    label = df['label'].values.tolist()
    tweets = df['tweet'].values.tolist()
    return tweets, label

tweets, label = load_as_list("train.csv")
print(tweets[:5])

x_train = tweets[:25570]
y_train = label[:25570]

x_test = tweets[25570:]
y_test = label[:25570:]


for i in range(len(x_train)):
    x_train[i] = ' '.join([word for word in x_train[i].split() if word not in cachedStopWords])
    x_train[i] = re.sub(r'[^\w\s]', '', x_train[i])

print(len(x_train))
print(len(label))
# print(len(vocab))
print(x_train[:5])
# print(vocab[:5])

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run', "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked", '  bihday your majesty', '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ', ' factsguide: society now    #motivation']
21602
21602
['user father dysfunctional selfish drags kids dysfunction run', 'user user thanks lyft credit cant use cause offer wheelchair vans pdx disapointed getthanked', 'bihday majesty', 'model love u take u time urð ðððð ððð', 'factsguide society motivation']


In [6]:
word_count = {}
for word in x_train:
    word_list = word.split()
    for sub_word in word_list:
        if sub_word in word_count:
            word_count[sub_word] += 1
        else:
            word_count[sub_word] = 1
print("executed")

word_count = {key:val for key, val in word_count.items() if val >= 30}
# print(word_count)
print(len(word_count))

executed
949


In [7]:
accepted_list = list(word_count.keys())

for word in x_train:
    word_list = word.split()
    final_word_list = [word for word in word_list if word in accepted_list]
    word = ' '.join(final_word_list)
    
print(x_train[:5])
print(len(tweets))
print(len(label))
    

['user father dysfunctional selfish drags kids dysfunction run', 'user user thanks lyft credit cant use cause offer wheelchair vans pdx disapointed getthanked', 'bihday majesty', 'model love u take u time urð ðððð ððð', 'factsguide society motivation']
21602
21602


In [8]:
vocab_size = 1200
encoded_docs = [one_hot(d, vocab_size) for d in x_train]
print(encoded_docs[:5])

# pad documents to a max length of 4 words
max_length = 512
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

[[886, 1073, 344, 166, 484, 887, 992, 177], [886, 886, 988, 1093, 486, 382, 798, 665, 38, 122, 839, 146, 999, 454], [520, 192], [435, 317, 386, 25, 386, 1187, 528, 262, 77], [359, 1162, 1044]]
21602


In [ ]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length))
model.add(LSTM(1024))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, np.array(y_train), epochs= 5, batch_size = 64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 32)           38400     
                                                                 
 lstm (LSTM)                 (None, 1024)              4329472   
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 4,368,897
Trainable params: 4,368,897
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
338/338 [==============================] - 10162s 30s/step - loss: 0.2643 - accuracy: 0.9288
Epoch 2/5
338/338 [==============================] - 10072s 30s/step - loss: 0.2537 - accuracy: 0.9309
Epoch 3/5
133/338 [==========>...................] - ETA: 1:40:47 - loss: 0.2548 - accuracy: 0.9302

In [ ]:
loss, accuracy = model.evaluate(padded_docs, np.array(y_train))
print('Accuracy: %f' % (accuracy*100))